In [1]:
import requests
import pandas as pd
import numpy as np
import plotly.express as px
import random

In [59]:
def last_price(market_id):

    url = f'https://www.buda.com/api/v2/markets/{market_id}/ticker'
    response = requests.get(url)
    ticker_info = response.json()
    return float(ticker_info['ticker']['last_price'][0])

def precio_ponderado_order(df, x):
    '''funcion calcula el precio ponderado si quiero ejecutar x cantidad
    df: dataframe con ordenes
    x: cantidad de cripto'''
    
    _df = df[df['Acum_vol']<= x].append(df[df['Acum_vol']>= x].head(1))
    _df.iloc[-1, _df.columns.get_loc('Acum_vol')] = x
    _df.iloc[-1, _df.columns.get_loc('volume')] = x - _df.iloc[-2]['Acum_vol']

    return ((_df['price'] * _df['volume']).sum()) / (_df['volume'].sum())

class Market():

    def __init__(self, market_id):
        self.id = market_id
        self.price = last_price(self.id)
        self.orders = False
        
    def __str__(self):
        return ('mercado %s Last Price: %s' %(self.id, self.price) )

    def last_price(self):
        self.price = last_price(self.id)
        return self.price
    
    def order_book(self, tipo=None):
        '''
        funcion obtiene libro de ordenes
        tipo: [asks, bids]. None entregara ambos libros
        '''
        market_id = self.id
        self.orders = pd.DataFrame()
        url = f'https://www.buda.com/api/v2/markets/{market_id}/order_book'
        response = requests.get(url)
        order_book = response.json()['order_book']
        columnas = ['price', 'volume']
        df_a = pd.DataFrame(order_book['asks'], columns=columnas).astype(float)
        df_a['tip'] = 'asks'
        df_a.set_index('tip', inplace=True)
        df_b = pd.DataFrame(order_book['bids'], columns=columnas).astype(float)
        df_b['tip'] = 'bids'
        df_b.set_index('tip', inplace=True)
        
        self.orders = pd.concat([df_a, df_b])

        if tipo == 'asks':
            return df_a
        elif tipo == 'bids':
            return df_b
        else:
            return self.orders

    def market_liquidity(self):
        
        self.order_book()
        
        # Suma Volumnes tottales agrupa en bid y en ask
        liqdty_idx = pd.pivot_table(self.orders, index=self.orders.index, values='volume', aggfunc=np.sum)
        # liqdty_idx['Precio_Ponderado'] = self.orders.groupby(self.orders.index).apply(lambda x: np.average(x['price'], weights=x['volume']))
        # Separo libros por tipo y ordeno por precio
        df_ask = self.orders.loc['asks'].sort_values('price')
        df_bid = self.orders.loc['bids'].sort_values('price', ascending=False)

        # creo columna de suma acumulada
        df_ask['Acum_vol'] = df_ask['volume'].cumsum()
        df_bid['Acum_vol'] = df_bid['volume'].cumsum()
        
        for x in np.linspace(0.1,1,10):
            # Creo rangos 10%, 20%... 100% del volumnes total
            x_ask = liqdty_idx['volume']['asks'] * x
            x_bid = liqdty_idx['volume']['bids'] * x
            # cAculo precio ponderado y guardo en liqdty_idx
            liqdty_idx['wght_price'+str(int(x*100)) +'%'] = [precio_ponderado_order(df_ask, x_ask), precio_ponderado_order(df_bid, x_bid)]

        return liqdty_idx

    def market_cap(self):

        self.order_book()

        # Volumne total * last price
        cap_a = self.orders.loc['asks']['volume'].sum() * self.last_price()
        cap_b = self.orders.loc['bids']['volume'].sum() * self.last_price()
        return ({'asks':cap_a, 'bids': cap_b})


    def cantidad_mercado_impacto(self, q, tipo = 'buy'):
        '''funcion 
        devuelve la cantidad q hay q vender/comprar para que el precio cambie en q%
        '''
        self.order_book()
        
        if tipo == 'buy':
            df = self.orders.loc['asks']
            df['Cumulative'] = df['volume'].cumsum()
            price_limit = self.price * (1 + q)
            df = df[df['price']>=price_limit]
        else:
            df = self.orders.loc['bids']
            df['Cumulative'] = df['volume'].cumsum()
            price_limit = self.price * (1 - q)
            df = df[df['price']<=price_limit]

        return df['Cumulative'].values[0]

    def market_depth(self, tipo='asks'):
        df_a = self.orders.loc[tipo]
        df_a['Cumulative'] = df_a['volume'].cumsum()
        df_a['change'] = df_a['price'] / self.price -1

        # df_b = self.orders.loc['bids']
        # df_b['Cumulative'] = df_b['volume'].cumsum()
        # df_b['change'] = df_b['price'] / self.price -1

        data = {}
        for i in np.linspace(0.05,1,20):
            if tipo == 'asks':
                mount = df_a[df_a['change']>=i]['Cumulative'].values[0]
            elif tipo =='bids':
                mount = df_a[df_a['change']<=-i]['Cumulative'].min()
            # data.append({tipo:{i:mount}})
            data[i]= [mount]
            # data.append({'bid':{i:mount_b}})

        return data

    def precio_fin_order(self, p, tipo='asks'):
        '''
        devuelve el precio final que quedara el mercado despues de ejecutar p unidades
        '''
        df = self.orders.loc[tipo]
        df['Cumulative'] = df['volume'].cumsum()
        
        df = df[df['Cumulative']>=p]
        price = df['price'].values[0]
        return {'price:': price, 'Change:': price /self.price -1}
        
class cartera():

    def __init__(self) -> None:
        
        self.creditos = pd.DataFrame(columns=['Prestamo', 'Colateral'])

    def add_credito(self, _prestamo, _colateral):

        self.creditos = self.creditos.append(pd.DataFrame([[_prestamo, _colateral]], columns=['Prestamo', 'Colateral']), ignore_index=True)

    def ltv(self, price):
        self.creditos['LTV'] = self.creditos['Prestamo'] / (self.creditos['Colateral'] * price)

    def crea_cartera(self, n, precio_med, std=0.1 ):
        df = pd.concat([pd.DataFrame(np.random.randint(1, 8, size=n) * 1000000, columns=['Prestamo']), pd.DataFrame(np.random.normal(loc=1, scale=std, size=n) * precio_med, columns=['Precio_I'])], axis=1)
        df['Colateral'] = df['Prestamo']/(df['Precio_I'] * 0.5)
        df.drop(columns=['Precio_I'], inplace=True)
        self.creditos = df

    def simulacion_crash(self, p, price):
        precio_crash = price * (1 - p) # price debe ser input
        df_sim = self.creditos
        # calculo nuevo ltv con caide del p%
        df_sim['LTV_fin'] = df_sim['Prestamo']/(df_sim['Colateral'] * price * (1 - p))
        df_sim = df_sim[df_sim['LTV_fin']>= .8]

        # principal_tarjet: 
        # amount: lo que tengo q liquidar
        df_sim['principal_target'] = (0.65 * df_sim['Colateral'] * precio_crash)
        df_sim['amount'] = (df_sim['Prestamo'] - df_sim['principal_target']) / (1-0.65)
        # df_sim['amount'] = df_sim[['Prestamo', 'amount']].apply(lambda x: x['Prestamo'] if x['Prestamo']<x['amount'] else x['amount'], axis=1)

        # como quedan los creditos despues de la liquidación
        # df_sim['principal_target'].sum()
        df_sim['Prestamo_after_liq'] = (df_sim['Prestamo'] - df_sim['amount'])
        df_sim['LTV_after_liq'] = (df_sim['Prestamo'] - df_sim['amount']) / (df_sim['Colateral'] * precio_crash -df_sim['amount'])
        self.cart_crash = df_sim
        self.cartera_to_liquidate = round(df_sim['amount'].sum())
        

In [3]:
btc = Market('btc-clp')
df_order = btc.order_book().loc['bids']

In [4]:
# definido una flag del 8% solo me puedo quedar con las ordenes hasta ese limite
flag = 0.08
df_order = df_order[df_order['price']>=btc.last_price() * (1-flag)]
df_order['Acumulado'] = df_order['volume'].cumsum()
df_order['total'] = df_order['price'] * df_order['volume']

In [5]:
max_liq_clp = round(df_order['total'].sum())
max_liq_btc = df_order['volume'].sum()
print('Maximo BTC a liquidar: '+ str(max_liq_btc)) # total que se puede liquidar en btc
print('Maximo CLP a liquidar: '+ str(max_liq_clp)) # maximo a liquidar en clp

Maximo BTC a liquidar: 7.519171620000001
Maximo CLP a liquidar: 345432667


In [ ]:
# cart = cartera()
# ltv_i = 0.5
# n = 250 # cantidad de créditos
# a_min = 1 # Prestamo minimo en millones
# a_max = 10 # Prestamo maximo en millones
# p_min = 40000000 # Precio minimo al q se obtiene un credito
# p_max = 60000000 # Precio maximo al que se pudo tomar un crédito

# for i in range(0, n):
#     prestamo = random.randint(a_min, a_max) * 1000000 # monto prestado entre 1 y 10 M
#     price = random.randrange(p_min, p_max, 100000) #precio aleatorio cuando se solicita un credito
#     colt = prestamo / (price * ltv_i) # colateral calculado en base al prestamo solicitado y el precio
#     cart.add_credito(prestamo, colt)


In [ ]:
# cart.ltv(btc.last_price())
# total_prestado = cart.creditos['Prestamo'].sum()
# cart.creditos

In [ ]:
# simular ltv con una caida del p%
# p = 0.3
# precio_crash = btc.last_price() * (1 - p)
# df_sim = cart.creditos
# # calculo nuevo ltv con caide del p%
# df_sim['LTV_fin'] = df_sim['Prestamo']/(df_sim['Colateral']* btc.last_price() * (1 - p))
# df_sim = df_sim[df_sim['LTV_fin']>= .8]

# # principal_tarjet: 
# # amount: lo que tengo q liquidar
# df_sim['principal_target'] = (0.65 * df_sim['Colateral'] * precio_crash)
# df_sim['amount'] = (df_sim['Prestamo'] - df_sim['principal_target']) / (1-0.65)

# # como quedan los creditos despues de la liquidación
# # df_sim['principal_target'].sum()
# df_sim['Prestamo_after_liq'] = (df_sim['Prestamo'] - df_sim['amount'])
# df_sim['LTV_after_liq'] = (df_sim['Prestamo'] - df_sim['amount']) / (df_sim['Colateral'] * precio_crash -df_sim['amount'])
# df_sim

In [6]:
# def simulacion_creditos(n, path, p, precio_inicial, precio_actual ):
#     resultado = pd.DataFrame(columns=['Total_prestamo','Volumen_mkt','Total_liquidado'])
#     ltv_i = 0.5
#     # n = 500 # cantidad de créditos
#     a_min = 1 # Prestamo minimo en millones
#     a_max = 8 # Prestamo maximo en millones
#     # path = 500 # cantidad de simulaciones
#     # p =.30
#     # precio_inicial = btc.last_price() # precio como medida central para la distribucion de precios
#     # precio_actual = btc.last_price()

#     for k in range(0,path):
#         cart = cartera()
#         price_data = np.random.normal(loc=1, scale=0.15, size=n) # ditribucion normal para calcular precio inicial de cada prestamo
#         for i in range(0, n):
#             prestamo = random.randint(a_min, a_max) * 1000000 # monto prestado entre 1 y 8 M
#             # price_i = random.randrange(p_min, p_max, 100000) #precio aleatorio cuando se solicita un credito
#             price_i = price_data[i] * precio_inicial # precio inicial del crédito, dado por el precio actual y distr normal
#             colt = prestamo / (price_i * ltv_i) # colateral calculado en base al prestamo solicitado y el precio
#             cart.add_credito(prestamo, colt)

#         cart.ltv(precio_actual)
#         total_prestado = cart.creditos['Prestamo'].sum() # todo lo q voy a prestar

#         cart.simulacion_crash(p, precio_actual)

#         resultado = resultado.append(pd.DataFrame([[total_prestado, max_liq_clp, cart.cartera_to_liquidate]], columns=['Total_prestamo','Volumen_mkt','Total_liquidado']), ignore_index=True)

#     resultado['Out_liquidity'] = resultado['Total_liquidado'] > resultado['Volumen_mkt']

#     return pd.DataFrame(np.array([[resultado['Total_prestamo'].mean(), resultado['Volumen_mkt'].mean(), resultado['Total_liquidado'].mean(),resultado['Out_liquidity'].sum()/path, p]]),columns=['Promedio prestado', 'Tamaño mercado', 'Colateral liquidado promedio', 'Relacion Quiebre de mercado', 'Caida'])


In [ ]:
# print('Promedio prestado: %s' %(resultado['Total_prestamo'].mean()))
# print('Tamaño mercado: %s' %(resultado['Volumen_mkt'].mean()))
# print('Colateral liquidado promedio: %s' %(resultado['Total_liquidado'].mean()))
# print('Veces Quiebre de mercado: %s' %(resultado['Out_liquidity'].sum()))

In [62]:
def simulacion_creditos(n, path, p, precio_inicial, precio_actual ):
    # n = 500 # cantidad de créditos
    # path = 500 # cantidad de simulaciones
    # p =.30
    # precio_inicial = btc.last_price() # precio como medida central para la distribucion de precios
    # precio_actual = btc.last_price() # precio que inicia la caida
    resultado = pd.DataFrame(columns=['Total_prestamo','Volumen_mkt','Total_liquidado'])
    # ltv_i = 0.5
    for _ in range(0, path):
        cart = cartera()
        cart.crea_cartera(n, precio_inicial, 0.12)
        cart.ltv(precio_actual)
        total_prestado = cart.creditos['Prestamo'].sum() # todo lo q voy a prestar

        cart.simulacion_crash(p, precio_actual)

        resultado = resultado.append(pd.DataFrame([[total_prestado, max_liq_clp, cart.cartera_to_liquidate]], columns=['Total_prestamo','Volumen_mkt','Total_liquidado']), ignore_index=True)

    resultado['Out_liquidity'] = resultado['Total_liquidado'] > resultado['Volumen_mkt']

    return pd.DataFrame(np.array([[resultado['Total_prestamo'].mean(), resultado['Volumen_mkt'].mean(), resultado['Total_liquidado'].mean(),resultado['Out_liquidity'].sum()/path, p]]),columns=['Promedio prestado', 'Tamaño mercado', 'Colateral liquidado promedio', 'Relacion Quiebre de mercado', 'Caida'])


In [70]:
simulaciones = pd.DataFrame(columns= ['Promedio prestado', 'Tamaño mercado', 'Colateral liquidado promedio', 'Relacion Quiebre de mercado', 'Caida'])
precio_sim = 48000000
precio_med = 49000000
for p in range(15, 36, 1):
    simulaciones = simulaciones.append(simulacion_creditos(700, 500, p/100, precio_med, precio_sim))

/var/folders/y5/6qkmm8_91817rs0kg5vpd5vm0000gn/T/ipykernel_9863/2674204502.py:173: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sim['principal_target'] = (0.65 * df_sim['Colateral'] * precio_crash)
/var/folders/y5/6qkmm8_91817rs0kg5vpd5vm0000gn/T/ipykernel_9863/2674204502.py:174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sim['amount'] = (df_sim['Prestamo'] - df_sim['principal_target']) / (1-0.65)
/var/folders/y5/6qkmm8_91817rs0kg5vpd5vm0000gn/T/ipykernel_9863/2674204502.py:179: SettingWithCopyWar

In [71]:
simulaciones

,Promedio prestado,Tamaño mercado,Colateral liquidado promedio,Relacion Quiebre de mercado,Caida
0,2.794896e+09,345432667.0,4.860468e+06,0.000,0.15
0,2.799006e+09,345432667.0,7.434168e+06,0.000,0.16
0,2.802252e+09,345432667.0,9.973607e+06,0.000,0.17
0,2.799832e+09,345432667.0,1.442908e+07,0.000,0.18
0,2.799878e+09,345432667.0,2.164196e+07,0.000,0.19
0,2.801100e+09,345432667.0,2.986429e+07,0.000,0.20
0,2.800600e+09,345432667.0,4.021435e+07,0.000,0.21
0,2.796930e+09,345432667.0,5.491951e+07,0.000,0.22
0,2.801202e+09,345432667.0,7.366695e+07,0.000,0.23
0,2.801294e+09,345432667.0,9.788901e+07,0.000,0.24


In [75]:
simulaciones['Relacion liquidado/prestado'] = simulaciones['Colateral liquidado promedio'] / simulaciones['Promedio prestado']
px.line(simulaciones, y='Relacion liquidado/prestado', x ='Caida')